#Bikeshare Analysis 2016-2018

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

In [0]:
#converting 2016 Q3 file to csv
read_file = pd.read_excel (r'/content/drive/My Drive/BikeshareData - City of Toronto/2016Q3.xlsx')
read_file.to_csv (r'/content/drive/My Drive/BikeshareData - City of Toronto/2016Q3.csv', index = None, header=True)

In [0]:
#converting 2016 Q4 file to csv
read_file = pd.read_excel (r'/content/drive/My Drive/BikeshareData - City of Toronto/2016Q4.xlsx')
read_file.to_csv (r'/content/drive/My Drive/BikeshareData - City of Toronto/2016Q4.csv', index = None, header=True)

In [0]:
df = pd.read_csv('/content/drive/My Drive/BikeshareData - City of Toronto/2016Q3.csv')

##Reading all Quarters into Dataframes 

In [0]:
df2 = pd.read_csv('/content/drive/My Drive/BikeshareData - City of Toronto/2016Q4.csv')

In [0]:
df3 = pd.read_csv('/content/drive/My Drive/BikeshareData - City of Toronto/2017Q1.csv')

In [0]:
df4 = pd.read_csv('/content/drive/My Drive/BikeshareData - City of Toronto/2017Q2.csv')

In [0]:
df5 = pd.read_csv('/content/drive/My Drive/BikeshareData - City of Toronto/2017Q3.csv')

In [0]:
df6 = pd.read_csv('/content/drive/My Drive/BikeshareData - City of Toronto/2017Q4.csv')

In [0]:
df7 = pd.read_csv('/content/drive/My Drive/BikeshareData - City of Toronto/2018Q1.csv')

In [0]:
df8 = pd.read_csv('/content/drive/My Drive/BikeshareData - City of Toronto/2018Q2.csv')

In [0]:
df9 = pd.read_csv('/content/drive/My Drive/BikeshareData - City of Toronto/2018Q3.csv')

In [0]:
df10 = pd.read_csv('/content/drive/My Drive/BikeshareData - City of Toronto/2018Q4.csv')

#Data Cleaning

##Approximately 6 different datetime formats, must convert to datetime objects


###2016 Quarter3

In [576]:
df.head()

,trip_id,trip_start_time,trip_stop_time,trip_duration_seconds,from_station_name,to_station_name,user_type
0,53279,2016-07-09 01:03:00,2016-07-09 01:15:00,714,Dundas St E / Regent Park Blvd,Danforth Ave / Ellerbeck St,Member
1,53394,2016-07-09 02:15:00,2016-07-09 02:22:00,417,Riverdale Park North (Broadview Ave),Dundas St E / Regent Park Blvd,Member
2,58314,2016-07-10 17:04:00,2016-07-10 17:36:00,1904,Dundas St E / Regent Park Blvd,Queen St W / Close Ave,Member
3,60784,2016-07-11 01:45:00,2016-07-11 01:58:00,784,Union Station,Dundas St E / Regent Park Blvd,Member
4,93164,2016-07-18 13:35:00,2016-07-18 13:42:00,443,Front St W / Blue Jays Way,Front St / Yonge St (Hockey Hall of Fame),Member


In [0]:
#converting start times to datetime
df['trip_start_time'] = pd.to_datetime(df['trip_start_time'],errors = 'coerce',format = '%Y-%m-%d %H:%M:%S')

In [0]:
#converting stop times to datetime
df['trip_stop_time'] = pd.to_datetime(df['trip_stop_time'],errors = 'coerce',format = '%Y-%m-%d %H:%M:%S')

In [50]:
#checking to see conversion was successful
df.dtypes

trip_id                           int64
trip_start_time          datetime64[ns]
trip_stop_time           datetime64[ns]
trip_duration_seconds             int64
from_station_name                object
to_station_name                  object
user_type                        object
dtype: object

In [51]:
#checking to see if datetime conversion cause any NaN values
df.isna().sum()

trip_id                  0
trip_start_time          0
trip_stop_time           0
trip_duration_seconds    0
from_station_name        3
to_station_name          2
user_type                0
dtype: int64

###2016 Quarter 4

Two different datetime formats in this dataset must convert one to the other to create datetime objects

'%Y-%d-%m %H:%M:%S'

'%d/%m/%Y %H:%M'

In [632]:
df2.head()

,trip_id,trip_start_time,trip_stop_time,trip_duration_seconds,from_station_name,to_station_name,user_type
0,462305,2016-10-01 00:00:00,2016-10-01 00:07:00,394,Queens Quay W / Dan Leckie Way,Fort York Blvd / Garrison Rd,Casual
1,462306,2016-10-01 00:00:00,2016-10-01 00:09:00,533,Sherbourne St / Wellesley St,Edward St / Yonge St,Member
2,462307,2016-10-01 00:00:00,2016-10-01 00:07:00,383,Queens Quay W / Dan Leckie Way,Fort York Blvd / Garrison Rd,Casual
3,462308,2016-10-01 00:01:00,2016-10-01 00:27:00,1557,Cherry St / Distillery Ln,Fort York Blvd / Capreol Crt,Casual
4,462309,2016-10-01 00:01:00,2016-10-01 00:27:00,1547,Cherry St / Distillery Ln,Fort York Blvd / Capreol Crt,Casual


Applying format changes to trip start time


In [0]:
#replace / with -
df2['trip_start_time'] = df2.trip_start_time.str.replace('/','-')

In [0]:
#split by first space to seperate date and time
startfix = df2.trip_start_time.str.split(' ',1,expand=True)

In [0]:
#set the date to original column
df2['trip_start_time'] = startfix[0]

In [0]:
#split by - for entries in wrong order having year at the end(%f-%f-%Y)
startfixed = df2.trip_start_time[df2.trip_start_time.str.contains('-201')].str.split('-',expand=True)

In [0]:
#re-arrange order
startfixed = startfixed[[2,0,1]]

In [0]:
#join by correct order
startfixed = startfixed.stack().groupby(level=0).apply('-'.join)

In [0]:
#replace values in dataframe
df2.loc[startfixed.index,'trip_start_time'] = startfixed

In [0]:
#set time to its own column
df2['StartTime'] = startfix[1]

In [0]:
#split by colons to seperate seconds
starttimefix = df2.StartTime.str.split(':',expand = True)

In [0]:
#keep hours and minutes while disregarding seconds and minutes
starttimefix = starttimefix[[0,1]]

In [0]:
starttimefix = starttimefix.stack().groupby(level=0).apply(':'.join)

In [0]:
#replace values in dataframe
df2.loc[starttimefix.index,'StartTime'] = starttimefix

In [0]:
#concatenating time and date back together
df2['trip_start_time'] = df2[['trip_start_time','StartTime']].agg(' '.join, axis=1)

In [0]:
#convert to datetime
df2['trip_start_time'] = pd.to_datetime(df2.trip_start_time,format = '%Y-%d-%m %H:%M')

In [0]:
del df2['StartTime']

Applying format changes to trip stop time

In [0]:
#replace / with -
df2['trip_stop_time'] = df2.trip_stop_time.str.replace('/','-')
#split by first space to seperate date and time
stopfix = df2.trip_stop_time.str.split(' ',1,expand=True)
#set the date to original column
df2['trip_stop_time'] = stopfix[0]
#split by - for entries in wrong order having year at the end(%f-%f-%Y)
stopfixed = df2.trip_stop_time[df2.trip_stop_time.str.contains('-201')].str.split('-',expand=True)
#re-arrange order
stopfixed = stopfixed[[2,0,1]]
#join by correct order
stopfixed = stopfixed.stack().groupby(level=0).apply('-'.join)
#replace values in dataframe
df2.loc[stopfixed.index,'trip_stop_time'] = stopfixed
#set time to its own column
df2['StopTime'] = stopfix[1]
#split by colons to seperate seconds
stoptimefix = df2.StopTime.str.split(':',expand = True)
#keep hours and minutes while disregarding seconds and minutes
stoptimefix = stoptimefix[[0,1]]
stoptimefix = stoptimefix.stack().groupby(level=0).apply(':'.join)
#replace values in dataframe
df2.loc[stoptimefix.index,'StopTime'] = stoptimefix
#concatenating time and date back together
df2['trip_stop_time'] = df2[['trip_stop_time','StopTime']].agg(' '.join, axis=1)
#convert to datetime
df2['trip_stop_time'] = pd.to_datetime(df2.trip_stop_time,format = '%Y-%d-%m %H:%M')
del df2['StopTime']

In [68]:
df2.isna().sum()

trip_id                  0
trip_start_time          0
trip_stop_time           0
trip_duration_seconds    0
from_station_name        2
to_station_name          2
user_type                0
dtype: int64

In [69]:
df2.dtypes

trip_id                           int64
trip_start_time          datetime64[ns]
trip_stop_time           datetime64[ns]
trip_duration_seconds             int64
from_station_name                object
to_station_name                  object
user_type                        object
dtype: object

###2017 Quarter 1

In [525]:
df3.head()

,trip_id,trip_start_time,trip_stop_time,trip_duration_seconds,from_station_id,from_station_name,to_station_id,to_station_name,user_type
0,712382,2017-01-01 00:00:00,1/1/2017 0:03,223,7051,Wellesley St E / Yonge St Green P,7089,Church St / Wood St,Member
1,712383,2017-01-01 00:00:00,1/1/2017 0:05,279,7143,Kendal Ave / Bernard Ave,7154,Bathurst Subway Station,Member
2,712384,2017-01-01 00:05:00,1/1/2017 0:29,1394,7113,Parliament St / Aberdeen Ave,7199,College St W / Markham St,Member
3,712385,2017-01-01 00:07:00,1/1/2017 0:21,826,7077,College Park South,7010,King St W / Spadina Ave,Member
4,712386,2017-01-01 00:08:00,1/1/2017 0:12,279,7079,McGill St / Church St,7047,University Ave / Gerrard St W,Member


In [0]:
df3['trip_start_time'] = pd.to_datetime(df3['trip_start_time'],errors = 'coerce',format = '%d/%m/%Y %H:%M')

In [0]:
df3['trip_stop_time'] = pd.to_datetime(df3['trip_stop_time'],errors = 'coerce',format = '%d/%m/%Y %H:%M')

In [634]:
df3.isna().sum()

trip_id                  0
trip_start_time          0
trip_stop_time           0
trip_duration_seconds    0
from_station_id          0
from_station_name        0
to_station_id            0
to_station_name          0
user_type                0
dtype: int64

In [635]:
df3.dtypes

trip_id                           int64
trip_start_time          datetime64[ns]
trip_stop_time           datetime64[ns]
trip_duration_seconds             int64
from_station_id                   int64
from_station_name                object
to_station_id                     int64
to_station_name                  object
user_type                        object
dtype: object

###2017 Quarter 2

In [529]:
df4.tail()

,trip_id,trip_start_time,trip_stop_time,trip_duration_seconds,from_station_id,from_station_name,to_station_id,to_station_name,user_type
333348,1253139,2017-06-30 23:59:00,1/7/2017 0:12,783,7163,Yonge St / Wood St,7118,King St W / Bay St (East Side),Member
333349,1253140,2017-06-30 23:59:00,1/7/2017 0:34,2081,7199,College St W / Markham St,7043,Queens Quay W / Lower Simcoe St,Casual
333350,1253141,2017-06-30 23:59:00,1/7/2017 0:03,227,7049,Queen St W / Portland St,7046,Niagara St / Richmond St W,Member
333351,1253142,2017-06-30 23:59:00,1/7/2017 0:34,2067,7199,College St W / Markham St,7043,Queens Quay W / Lower Simcoe St,Casual
333352,1253144,2017-06-30 23:59:00,1/7/2017 0:26,1623,7006,Bay St / College St (East Side),7093,Danforth Ave / Gough Ave,Member


In [0]:
df4['trip_start_time'] = pd.to_datetime(df4['trip_start_time'],errors = 'coerce',format = '%d/%m/%Y %H:%M')

In [0]:
df4['trip_stop_time'] = pd.to_datetime(df4['trip_stop_time'],errors = 'coerce',format = '%d/%m/%Y %H:%M')

In [74]:
df4.isna().sum()

trip_id                  0
trip_start_time          0
trip_stop_time           0
trip_duration_seconds    0
from_station_id          0
from_station_name        0
to_station_id            0
to_station_name          0
user_type                0
dtype: int64

In [75]:
df4.dtypes

trip_id                           int64
trip_start_time          datetime64[ns]
trip_stop_time           datetime64[ns]
trip_duration_seconds             int64
from_station_id                   int64
from_station_name                object
to_station_id                     int64
to_station_name                  object
user_type                        object
dtype: object

###2017 Quarter 3

In [533]:
df5.head()

,trip_id,trip_start_time,trip_stop_time,trip_duration_seconds,from_station_name,to_station_name,user_type
0,1253914,2017-07-01 00:00:00,7/1/2017 0:15,910,Princess St / Adelaide St E,424 Wellington St W,Member
1,1253915,2017-07-01 00:01:00,7/1/2017 0:15,837,Fort York Blvd / Capreol Crt,HTO Park (Queens Quay W),Casual
2,1253916,2017-07-01 00:01:00,7/1/2017 0:14,786,Fort York Blvd / Capreol Crt,HTO Park (Queens Quay W),Casual
3,1253917,2017-07-01 00:01:00,7/1/2017 0:25,1420,Elizabeth St / Edward St (Bus Terminal),Boston Ave / Queen St E,Casual
4,1253918,2017-07-01 00:01:00,7/1/2017 0:25,1437,Elizabeth St / Edward St (Bus Terminal),Boston Ave / Queen St E,Casual


In [0]:
df5['trip_start_time'] = pd.to_datetime(df5['trip_start_time'],errors = 'coerce',format = '%m/%d/%Y %H:%M')

In [0]:
df5['trip_stop_time'] = pd.to_datetime(df5['trip_stop_time'],errors = 'coerce',format = '%m/%d/%Y %H:%M')

In [640]:
df5.isna().sum()

trip_id                  0
trip_start_time          0
trip_stop_time           0
trip_duration_seconds    0
from_station_name        0
to_station_name          0
user_type                0
dtype: int64

In [641]:
df5.dtypes

trip_id                           int64
trip_start_time          datetime64[ns]
trip_stop_time           datetime64[ns]
trip_duration_seconds             int64
from_station_name                object
to_station_name                  object
user_type                        object
dtype: object

###2017 Quarter 4

In [537]:
df6.head()

,trip_id,trip_start_time,trip_stop_time,trip_duration_seconds,from_station_name,to_station_name,user_type
0,1971685,2017-10-01 00:00:01,10/01/17 00:14:10,849,Queen St W / James St,Taddle Creek Park,Member
1,1971686,2017-10-01 00:00:01,10/01/17 00:00:27,26,Stewart St / Bathurst St - SMART,Stewart St / Bathurst St - SMART,Casual
2,1971687,2017-10-01 00:00:22,10/01/17 00:22:12,1310,Hayter St / Laplante Ave,Queen St W / Portland St,Member
3,1971688,2017-10-01 00:00:38,10/01/17 00:11:09,631,Beverly St / College St,Beverly St / Dundas St W,Member
4,1971689,2017-10-01 00:00:55,10/01/17 00:20:53,1198,Thompson St / Broadview Ave - SMART,Thompson St / Broadview Ave - SMART,Member


In [0]:
df6['trip_start_time'] = pd.to_datetime(df6['trip_start_time'],errors = 'coerce',format = '%m/%d/%y %H:%M:%S')

In [0]:
df6['trip_stop_time'] = pd.to_datetime(df6['trip_stop_time'],errors = 'coerce',format = '%m/%d/%y %H:%M:%S')

In [654]:
df6.isna().sum()

trip_id                  0
trip_start_time          0
trip_stop_time           1
trip_duration_seconds    0
from_station_name        0
to_station_name          1
user_type                0
dtype: int64

In [655]:
df6.dtypes

trip_id                           int64
trip_start_time          datetime64[ns]
trip_stop_time           datetime64[ns]
trip_duration_seconds             int64
from_station_name                object
to_station_name                  object
user_type                        object
dtype: object

###2018 Quarter 1

In [540]:
df7.head()

,trip_id,trip_duration_seconds,from_station_id,trip_start_time,from_station_name,trip_stop_time,to_station_id,to_station_name,user_type
0,2383648,393,7018,2018-01-01 00:47:00,Bremner Blvd / Rees St,1/1/2018 0:54,7176,Bathurst St / Fort York Blvd,Annual Member
1,2383649,625,7184,2018-01-01 00:52:00,Ossington Ave / College St,1/1/2018 1:03,7191,Central Tech (Harbord St),Annual Member
2,2383650,233,7235,2018-01-01 00:55:00,Bay St / College St (West Side) - SMART,1/1/2018 0:59,7021,Bay St / Albert St,Annual Member
3,2383651,1138,7202,2018-01-01 00:57:00,Queen St W / York St (City Hall),1/1/2018 1:16,7020,Phoebe St / Spadina Ave,Annual Member
4,2383652,703,7004,2018-01-01 01:00:00,University Ave / Elm St,1/1/2018 1:12,7060,Princess St / Adelaide St E,Annual Member


In [0]:
df7['trip_start_time'] = pd.to_datetime(df7['trip_start_time'],errors = 'coerce',format = '%m/%d/%Y %H:%M')

In [0]:
df7['trip_stop_time'] = pd.to_datetime(df7['trip_stop_time'],errors = 'coerce',format = '%m/%d/%Y %H:%M')

In [657]:
df7.isna().sum()

trip_id                  0
trip_duration_seconds    0
from_station_id          0
trip_start_time          0
from_station_name        0
trip_stop_time           0
to_station_id            0
to_station_name          0
user_type                0
dtype: int64

In [658]:
df7.dtypes

trip_id                           int64
trip_duration_seconds             int64
from_station_id                   int64
trip_start_time          datetime64[ns]
from_station_name                object
trip_stop_time           datetime64[ns]
to_station_id                     int64
to_station_name                  object
user_type                        object
dtype: object

###2018 Quarter 2

In [543]:
df8.head()

,trip_id,trip_duration_seconds,from_station_id,trip_start_time,from_station_name,trip_stop_time,to_station_id,to_station_name,user_type
0,2605025,1666,7000,2018-04-01 00:01:00,Fort York Blvd / Capreol Ct,4/1/2018 0:29,7150,Dufferin St / Sylvan Av (Dufferin Grove Park),Annual Member
1,2605024,247,7051,2018-04-01 00:01:00,Wellesley St E / Yonge St (Green P),4/1/2018 0:05,7042,Sherbourne St / Wellesley St E,Annual Member
2,2605026,2032,7062,2018-04-01 00:03:00,University Ave / College St (West),4/1/2018 0:37,7056,Parliament St / Gerrard St,Annual Member
3,2605027,531,7063,2018-04-01 00:03:00,Queen's Park / Bloor St W,4/1/2018 0:12,7143,Kendal Ave / Bernard Ave,Annual Member
4,2605028,642,7199,2018-04-01 00:05:00,College St / Markham St,4/1/2018 0:15,7004,University Ave / Elm St,Annual Member


In [0]:
df8['trip_start_time'] = pd.to_datetime(df8['trip_start_time'],errors = 'coerce',format = '%m/%d/%Y %H:%M')

In [0]:
df8['trip_stop_time'] = pd.to_datetime(df8['trip_stop_time'],errors = 'coerce',format = '%m/%d/%Y %H:%M')

In [660]:
df8.isna().sum()

trip_id                  0
trip_duration_seconds    0
from_station_id          0
trip_start_time          0
from_station_name        0
trip_stop_time           0
to_station_id            0
to_station_name          0
user_type                0
dtype: int64

In [661]:
df8.dtypes

trip_id                           int64
trip_duration_seconds             int64
from_station_id                   int64
trip_start_time          datetime64[ns]
from_station_name                object
trip_stop_time           datetime64[ns]
to_station_id                     int64
to_station_name                  object
user_type                        object
dtype: object

###2018 Quarter 3

In [546]:
df9.head()

,trip_id,trip_duration_seconds,from_station_id,trip_start_time,from_station_name,trip_stop_time,to_station_id,to_station_name,user_type
0,3244396,528,7254,2018-07-01 00:00:00,Borden St / Bloor St W - SMART,7/1/2018 0:08,7140,Macpherson Ave / Spadina Rd,Annual Member
1,3244398,570,7031,2018-07-01 00:00:00,Jarvis St / Isabella St,7/1/2018 0:09,7167,Queen St E / George St (Moss Park),Annual Member
2,3244397,368,7286,2018-07-01 00:00:00,Gerrard St E / Broadview - SMART,7/1/2018 0:06,7286,Gerrard St E / Broadview - SMART,Casual Member
3,3244399,1147,7248,2018-07-01 00:00:00,Baldwin Ave / Spadina Ave - SMART,7/1/2018 0:19,7020,Phoebe St / Spadina Ave,Annual Member
4,3244400,730,7207,2018-07-01 00:02:00,Dundas St W / Crawford St,7/1/2018 0:14,7000,Fort York Blvd / Capreol Ct,Annual Member


In [0]:
df9['trip_start_time'] = pd.to_datetime(df9['trip_start_time'],errors = 'coerce',format = '%m/%d/%Y %H:%M')

In [0]:
df9['trip_stop_time'] = pd.to_datetime(df9['trip_stop_time'],errors = 'coerce',format = '%m/%d/%Y %H:%M')

In [663]:
df9.isna().sum()

trip_id                  0
trip_duration_seconds    0
from_station_id          0
trip_start_time          0
from_station_name        0
trip_stop_time           0
to_station_id            0
to_station_name          0
user_type                0
dtype: int64

In [664]:
df9.dtypes

trip_id                           int64
trip_duration_seconds             int64
from_station_id                   int64
trip_start_time          datetime64[ns]
from_station_name                object
trip_stop_time           datetime64[ns]
to_station_id                     int64
to_station_name                  object
user_type                        object
dtype: object

###2018 Quarter 4

In [549]:
df10.head()

,trip_id,trip_duration_seconds,from_station_id,trip_start_time,from_station_name,trip_stop_time,to_station_id,to_station_name,user_type
0,4158592,749,7061,2018-10-01 00:01:00,Dalton Rd / Bloor St W,10/1/2018 0:14,7042,Sherbourne St / Wellesley St E,Annual Member
1,4158593,433,7003,2018-10-01 00:06:00,Madison Ave / Bloor St W,10/1/2018 0:13,7280,Charles St E / Jarvis St - SMART,Annual Member
2,4158594,285,7024,2018-10-01 00:14:00,Dundonald St / Church St,10/1/2018 0:19,7028,Gould St / Mutual St,Annual Member
3,4158595,150,7190,2018-10-01 00:16:00,St. George St / Hoskin Ave,10/1/2018 0:18,7161,Beverley St / College St,Annual Member
4,4158596,744,7265,2018-10-01 00:21:00,Wallace Ave / Symington Ave - SMART,10/1/2018 0:33,7136,Queen St W / Close Ave,Annual Member


In [0]:
df10['trip_start_time'] = pd.to_datetime(df10['trip_start_time'],errors = 'coerce',format = '%m/%d/%Y %H:%M')

In [0]:
df10['trip_stop_time'] = pd.to_datetime(df10['trip_stop_time'],errors = 'coerce',format = '%m/%d/%Y %H:%M')

In [666]:
df10.isna().sum()

trip_id                  0
trip_duration_seconds    0
from_station_id          0
trip_start_time          0
from_station_name        0
trip_stop_time           0
to_station_id            0
to_station_name          0
user_type                0
dtype: int64

In [667]:
df10.dtypes

trip_id                           int64
trip_duration_seconds             int64
from_station_id                   int64
trip_start_time          datetime64[ns]
from_station_name                object
trip_stop_time           datetime64[ns]
to_station_id                     int64
to_station_name                  object
user_type                        object
dtype: object

##Merging all Dataframes into 1

In [504]:
#looking for differences in columns of each dataframe 
df.columns

Index(['trip_id', 'trip_start_time', 'trip_stop_time', 'trip_duration_seconds',
       'from_station_name', 'to_station_name', 'user_type'],
      dtype='object')

In [505]:
df2.columns

Index(['trip_id', 'trip_start_time', 'trip_stop_time', 'trip_duration_seconds',
       'from_station_name', 'to_station_name', 'user_type'],
      dtype='object')

In [506]:
df3.columns

Index(['trip_id', 'trip_start_time', 'trip_stop_time', 'trip_duration_seconds',
       'from_station_id', 'from_station_name', 'to_station_id',
       'to_station_name', 'user_type'],
      dtype='object')

In [507]:
df4.columns

Index(['trip_id', 'trip_start_time', 'trip_stop_time', 'trip_duration_seconds',
       'from_station_id', 'from_station_name', 'to_station_id',
       'to_station_name', 'user_type'],
      dtype='object')

In [508]:
df5.columns

Index(['trip_id', 'trip_start_time', 'trip_stop_time', 'trip_duration_seconds',
       'from_station_name', 'to_station_name', 'user_type'],
      dtype='object')

In [509]:
df6.columns

Index(['trip_id', 'trip_start_time', 'trip_stop_time', 'trip_duration_seconds',
       'from_station_name', 'to_station_name', 'user_type'],
      dtype='object')

In [510]:
df7.columns

Index(['trip_id', 'trip_duration_seconds', 'from_station_id',
       'trip_start_time', 'from_station_name', 'trip_stop_time',
       'to_station_id', 'to_station_name', 'user_type'],
      dtype='object')

In [511]:
df8.columns

Index(['trip_id', 'trip_duration_seconds', 'from_station_id',
       'trip_start_time', 'from_station_name', 'trip_stop_time',
       'to_station_id', 'to_station_name', 'user_type'],
      dtype='object')

In [512]:
df9.columns

Index(['trip_id', 'trip_duration_seconds', 'from_station_id',
       'trip_start_time', 'from_station_name', 'trip_stop_time',
       'to_station_id', 'to_station_name', 'user_type'],
      dtype='object')

In [513]:
df10.columns

Index(['trip_id', 'trip_duration_seconds', 'from_station_id',
       'trip_start_time', 'from_station_name', 'trip_stop_time',
       'to_station_id', 'to_station_name', 'user_type'],
      dtype='object')

**Datasets 3,4,7,8,9,10 have two additional columns being from_station_id and to_station_id**

In [0]:
frames = [df, df2, df3,df4,df5,df6,df7,df8,df9,df10]


In [0]:
#putting all dataframes together
bikeshare_df = pd.concat(frames)

In [0]:
id_list = frames_df['from_station_id'].unique()

In [0]:
id2_list = frames_df['to_station_id'].unique()

In [28]:
len(id2_list)

360

In [48]:
bikeshare_df.shape

(4000855, 9)

In [0]:
#dropping trip id to determine duplicates in data
bikeshare_df = bikeshare_df.drop(['trip_id'],axis = 1)

In [50]:
#determining number of duplicate rows
bikeshare_df.duplicated().sum()

7501

In [0]:
# dropping duplicated rows
bikeshare_df.drop_duplicates(keep=False,inplace=True)

In [52]:
#checking duplicates again
bikeshare_df.duplicated().sum()

0

In [53]:
bikeshare_df.shape

(3985860, 8)

In [54]:
bikeshare_df.reset_index(drop =True)

,trip_start_time,trip_stop_time,trip_duration_seconds,from_station_name,to_station_name,user_type,from_station_id,to_station_id
0,2016-07-09 01:03:00,2016-07-09 01:15:00,714,Dundas St E / Regent Park Blvd,Danforth Ave / Ellerbeck St,Member,NaN,NaN
1,2016-07-09 02:15:00,2016-07-09 02:22:00,417,Riverdale Park North (Broadview Ave),Dundas St E / Regent Park Blvd,Member,NaN,NaN
2,2016-07-10 17:04:00,2016-07-10 17:36:00,1904,Dundas St E / Regent Park Blvd,Queen St W / Close Ave,Member,NaN,NaN
3,2016-07-11 01:45:00,2016-07-11 01:58:00,784,Union Station,Dundas St E / Regent Park Blvd,Member,NaN,NaN
4,2016-07-18 13:35:00,2016-07-18 13:42:00,443,Front St W / Blue Jays Way,Front St / Yonge St (Hockey Hall of Fame),Member,NaN,NaN
...,...,...,...,...,...,...,...,...
3985855,2018-12-31 23:43:00,2018-12-31 23:49:00,379,Danforth Ave / Coxwell Ave,Donlands Station,Annual Member,7088.0,7091.0
3985856,2018-12-31 23:45:00,2018-12-31 23:50:00,306,Bay St / Wellesley St W,Jarvis St / Isabella St,Annual Member,7030.0,7031.0
3985857,2018-12-31 23:49:00,2018-12-31 23:55:00,340,Phoebe St / Spadina Ave,Fort York Blvd / Capreol Ct,Annual Member,7020.0,7000.0
3985858,2018-12-31 23:52:00,2019-01-01 00:17:00,1466,Sherbourne St / Carlton St (Allan Gardens),Toronto Eaton Centre (Yonge St),Annual Member,7014.0,7269.0


##Loading Station Data from Bikeshare API Endpoint

need latitude and longitude coordinates

In [55]:
pip install fuzzywuzzy

In [56]:
from datetime import timedelta
from fuzzywuzzy import fuzz

import matplotlib.pyplot as plt
import pandas as pd
import requests
import seaborn as sns

import json
import os


r = requests.get('https://tor.publicbikesystem.net/ube/gbfs/v1/en/station_information')

bikeshare_stations = pd.DataFrame(json.loads(r.content)['data']['stations'])[['station_id', 'name', 'lat', 'lon']].astype({
    'station_id': 'float64',
})

bikeshare_stations.to_csv('/content/drive/My Drive/BikeshareData - City of Toronto/bikeshare_stations.csv', index=False)

/usr/local/lib/python3.6/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
bikestations_df = pd.read_csv(r'/content/drive/My Drive/BikeshareData - City of Toronto/bikeshare_stations.csv')

In [159]:
bikestations_df.head()

,station_id,name,lat,lon
0,7000.0,Fort York Blvd / Capreol Ct,43.639832,-79.395954
1,7001.0,Lower Jarvis St / The Esplanade,43.647992,-79.370907
2,7002.0,St. George St / Bloor St W,43.667333,-79.399429
3,7003.0,Madison Ave / Bloor St W,43.667158,-79.402761
4,7004.0,University Ave / Elm St,43.656518,-79.389099


###Missing station Id from 6 of the 10 quarters

must match the latitude and longitude based on id therefore need to match station ids to names from station data set

In [0]:
#creating dataframe containing both start and end station names
stations_start = bikeshare_df[['from_station_id', 'from_station_name']]
stations_end = bikeshare_df[['to_station_id', 'to_station_name']]
stations_start.columns = stations_end.columns = ['station_id', 'name']

# Extracts the unique station ID and name combination from the from_station and to_station columns
stations = pd.concat([stations_start, stations_end]).dropna(how='all').drop_duplicates().reset_index(drop=True)

In [59]:
# Separate the stations without station IDs
no_ids = stations[stations['station_id'].isnull()]
for idx, miss in no_ids.iterrows():
    max_score = 0
    
    # Compare the similarity of the station without ID to each station in the API data
    for i, exist in bikeshare_stations[['station_id', 'name']].iterrows():
        score = fuzz.ratio(miss['name'], exist['name'])
        
        if score > 80 and score > max_score:
            max_score = score
            no_ids.at[idx, 'station_id'] = exist['station_id']
    
    # Warn if the station was not able to be matched
    if max_score <= 80:
        print('WARN: {0} station could not be matched to an existing station'.format(miss['name']))
        
# Remove all stations that were not matched
no_ids = no_ids.dropna()

WARN: Victoria St / Gould St (Ryerson University) station could not be matched to an existing station
WARN: Bloor St / Brunswick Ave station could not be matched to an existing station
WARN: Bay St / Bloor St W station could not be matched to an existing station
WARN: Bremner Blvd / Spadina Ave station could not be matched to an existing station
WARN: Dockside Dr / Queens Quay E (Sugar Beach) station could not be matched to an existing station
WARN: Temperance St / Yonge St station could not be matched to an existing station
WARN: Ontario Place Blvd / Remembrance Dr station could not be matched to an existing station
WARN: Landsdowne Subway Green P station could not be matched to an existing station
WARN: Bathurst St / Queens Quay W station could not be matched to an existing station
WARN: Bloor GO / UP Station/ Rail Path station could not be matched to an existing station
WARN: Stephenson Ave / Main St station could not be matched to an existing station
WARN: Woodbine Subway Green P (

In [0]:
#concatenating station data and merging with Bikeshare API Data
stations = pd.concat([stations[~stations['station_id'].isnull()], no_ids])\
             .merge(bikeshare_stations[['station_id', 'lat', 'lon']], how='inner', on='station_id')\
             .drop_duplicates()

In [107]:
stations.head()

,station_id,name,lat,lon
0,7051.0,Wellesley St E / Yonge St Green P,43.665060,-79.383570
1,7051.0,Wellesley St E / Yonge St (Green P),43.665060,-79.383570
4,7143.0,Kendal Ave / Bernard Ave,43.671513,-79.408317
6,7113.0,Parliament St / Aberdeen Ave,43.665278,-79.368333
8,7077.0,College Park South,43.659777,-79.382767


In [0]:
#matching stations data frame to bikeshare dataframe based on station names
bikeshare_df = bikeshare_df.merge(stations, how='inner', left_on='from_station_name', right_on='name') \
       .merge(stations, how='inner', left_on='to_station_name', right_on='name', suffixes=['_from', '_to']) \
       .drop_duplicates()

In [109]:
bikeshare_df.isna().sum()

trip_start_time                0
trip_stop_time                 0
trip_duration_seconds          0
from_station_name              0
to_station_name                0
user_type                      0
from_station_id          1602151
to_station_id            1602151
station_id_from                0
name_from                      0
lat_from                       0
lon_from                       0
station_id_to                  0
name_to                        0
lat_to                         0
lon_to                         0
dtype: int64

In [65]:
bikeshare_df.shape

(3946860, 12)

In [114]:
bikeshare_df.head()

,trip_start_time,trip_stop_time,trip_duration_seconds,user_type,station_id_from,name_from,lat_from,lon_from,station_id_to,name_to,lat_to,lon_to
0,2016-07-09 01:03:00,2016-07-09 01:15:00,714,Member,7100.0,Dundas St E / Regent Park Blvd,43.660207,-79.361275,7095.0,Danforth Ave / Ellerbeck St,43.677076,-79.35667
1,2016-09-03 22:37:00,2016-09-03 22:48:00,685,Member,7100.0,Dundas St E / Regent Park Blvd,43.660207,-79.361275,7095.0,Danforth Ave / Ellerbeck St,43.677076,-79.35667
2,2016-08-05 16:21:00,2016-08-05 16:33:00,718,Member,7100.0,Dundas St E / Regent Park Blvd,43.660207,-79.361275,7095.0,Danforth Ave / Ellerbeck St,43.677076,-79.35667
3,2016-09-23 10:15:00,2016-09-23 10:28:00,792,Member,7100.0,Dundas St E / Regent Park Blvd,43.660207,-79.361275,7095.0,Danforth Ave / Ellerbeck St,43.677076,-79.35667
4,2016-10-15 13:40:00,2016-10-15 13:51:00,653,Member,7100.0,Dundas St E / Regent Park Blvd,43.660207,-79.361275,7095.0,Danforth Ave / Ellerbeck St,43.677076,-79.35667


In [0]:
#no longer need from_station_name, to_station_name, from_station_id, to_station_id columns
del bikeshare_df['from_station_id']
del bikeshare_df['from_station_name']
del bikeshare_df['to_station_id']
del bikeshare_df['to_station_name']

##Removing Trip Less than 1 Minute, Assuming they are false trips

In [0]:
bikeshare_df = bikeshare_df[(bikeshare_df['trip_duration_seconds']>=60)]

In [0]:
#export as csv
export_csv = bikeshare_df.to_csv(r'/content/drive/My Drive/BikeshareData - City of Toronto/bikeshare_df.csv',index = False)

In [0]:
bikeshare_df = pd.read_csv(r'/content/drive/My Drive/BikeshareData - City of Toronto/bikeshare_df.csv',)

#Feature Engineering

In [0]:
#when reading csv back into notebook, must convert tip start time and stop time back to datetime
bikeshare_df['trip_start_time'] = pd.to_datetime(bikeshare_df['trip_start_time'],errors='coerce')
bikeshare_df['trip_stop_time'] = pd.to_datetime(bikeshare_df['trip_stop_time'],errors='coerce')

In [0]:
#creating columns to display hour of day for start and stop times
bikeshare_df['StartHour'] = bikeshare_df['trip_start_time'].dt.hour
bikeshare_df['StopHour'] = bikeshare_df['trip_stop_time'].dt.hour

In [0]:
#creating column for quarter of the year
bikeshare_df['Quarter'] = bikeshare_df['trip_start_time'].dt.quarter

In [0]:
#creating columns for month, dayofweek, daynumber, year, 
bikeshare_df['Year'] = bikeshare_df['trip_start_time'].dt.year
bikeshare_df['Month'] = bikeshare_df['trip_start_time'].dt.month
bikeshare_df['DayofWeek'] = bikeshare_df['trip_start_time'].dt.dayofweek
bikeshare_df['DayNumber'] = bikeshare_df['trip_start_time'].dt.day


In [0]:
#applying names to day of week numbers
weekdaynamedict = {0:'Monday',1:'Tuesday',2:'Wednesday',3:'Thursday',4:'Friday',5:'Saturday',6:'Sunday'}
bikeshare_df['DayofWeek'] = bikeshare_df['DayofWeek'].map(weekdaynamedict)

In [0]:
#calculating extra usage fees
bikeshare_df['ExtraFees'] = ((bikeshare_df['trip_duration_seconds'] - 1800)/1800)*4

In [0]:
#setting usage fees that are negative to zero
bikeshare_df.loc[bikeshare_df['ExtraFees']<=0,'ExtraFees'] = 0

In [0]:
#converting to whole numbers
bikeshare_df['ExtraFees'] = bikeshare_df['ExtraFees'].astype(int)

##Putting in Route IDs

In [0]:
#creating new column combining start and end station ids
bikeshare_df['Route Id'] = bikeshare_df.apply(lambda x: '{0}-{1}'.format(int(x['station_id_from']), int(x['station_id_to'])), axis=1)

In [0]:
#converting latitude and longitude columns to numeric types
bikeshare_df['lat_to'] = pd.to_numeric(bikeshare_df['lat_to'])
bikeshare_df['lat_from'] = pd.to_numeric(bikeshare_df['lat_from'])
bikeshare_df['lon_to'] = pd.to_numeric(bikeshare_df['lon_to'])
bikeshare_df['lon_from'] = pd.to_numeric(bikeshare_df['lon_from'])

In [75]:
bikeshare_df[bikeshare_df['Year'] == 2000]

,trip_start_time,trip_stop_time,trip_duration_seconds,user_type,station_id_from,name_from,lat_from,lon_from,station_id_to,name_to,lat_to,lon_to,StartHour,StopHour,Quarter,Year,Month,DayofWeek,DayNumber,ExtraFees,Route Id
91802,2000-01-01 14:40:00,2000-01-01 15:44:00,3794,Member,7048.0,Front St / Yonge St (Hockey Hall of Fame),43.646144,-79.377962,7048.0,Front St / Yonge St (Hockey Hall of Fame),43.646144,-79.377962,14,15,1,2000,1,Saturday,1,4,7048-7048


In [0]:
#getting rid of row with incorrect date 
bikeshare_df = bikeshare_df.drop([91802])

In [77]:
bikeshare_df.reset_index(drop = True)

,trip_start_time,trip_stop_time,trip_duration_seconds,user_type,station_id_from,name_from,lat_from,lon_from,station_id_to,name_to,lat_to,lon_to,StartHour,StopHour,Quarter,Year,Month,DayofWeek,DayNumber,ExtraFees,Route Id
0,2016-07-09 01:03:00,2016-07-09 01:15:00,714,Member,7100.0,Dundas St E / Regent Park Blvd,43.660207,-79.361275,7095.0,Danforth Ave / Ellerbeck St,43.677076,-79.35667,1,1,3,2016,7,Saturday,9,0,7100-7095
1,2016-09-03 22:37:00,2016-09-03 22:48:00,685,Member,7100.0,Dundas St E / Regent Park Blvd,43.660207,-79.361275,7095.0,Danforth Ave / Ellerbeck St,43.677076,-79.35667,22,22,3,2016,9,Saturday,3,0,7100-7095
2,2016-08-05 16:21:00,2016-08-05 16:33:00,718,Member,7100.0,Dundas St E / Regent Park Blvd,43.660207,-79.361275,7095.0,Danforth Ave / Ellerbeck St,43.677076,-79.35667,16,16,3,2016,8,Friday,5,0,7100-7095
3,2016-09-23 10:15:00,2016-09-23 10:28:00,792,Member,7100.0,Dundas St E / Regent Park Blvd,43.660207,-79.361275,7095.0,Danforth Ave / Ellerbeck St,43.677076,-79.35667,10,10,3,2016,9,Friday,23,0,7100-7095
4,2016-10-15 13:40:00,2016-10-15 13:51:00,653,Member,7100.0,Dundas St E / Regent Park Blvd,43.660207,-79.361275,7095.0,Danforth Ave / Ellerbeck St,43.677076,-79.35667,13,13,4,2016,10,Saturday,15,0,7100-7095
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3946854,2018-10-19 15:20:00,2018-10-19 15:40:00,1215,Annual Member,7380.0,Erskine Ave / Yonge St SMART,43.711215,-79.398940,7379.0,Lawton Blvd / Yonge St,43.691067,-79.39535,15,15,4,2018,10,Friday,19,0,7380-7379
3946855,2018-10-23 17:58:00,2018-10-23 18:05:00,448,Annual Member,7380.0,Erskine Ave / Yonge St SMART,43.711215,-79.398940,7379.0,Lawton Blvd / Yonge St,43.691067,-79.39535,17,18,4,2018,10,Tuesday,23,0,7380-7379
3946856,2018-11-13 15:16:00,2018-11-13 15:39:00,1372,Annual Member,7380.0,Erskine Ave / Yonge St SMART,43.711215,-79.398940,7379.0,Lawton Blvd / Yonge St,43.691067,-79.39535,15,15,4,2018,11,Tuesday,13,0,7380-7379
3946857,2018-09-23 16:46:00,2018-09-23 17:09:00,1375,Annual Member,7384.0,Cumberland Ave / Bay St SMART,43.670454,-79.390140,7379.0,Lawton Blvd / Yonge St,43.691067,-79.39535,16,17,3,2018,9,Sunday,23,0,7384-7379


In [0]:
#making station names under same id the same
bikeshare_df.loc[bikeshare_df['name_from'] == 'Wellesley St E / Yonge St (Green P)','name_from'] = 'Wellesley St E / Yonge St Green P'
bikeshare_df.loc[bikeshare_df['name_to'] == 'Wellesley St E / Yonge St (Green P)','name_to'] = 'Wellesley St E / Yonge St Green P'
bikeshare_df.loc[bikeshare_df['name_from'] == 'Bathurst St/Queens Quay(Billy Bishop Airport)','name_from'] = 'Bathurst St / Queens Quay W'
bikeshare_df.loc[bikeshare_df['name_to'] == 'Bathurst St/Queens Quay(Billy Bishop Airport)','name_to'] = 'Bathurst St / Queens Quay W'
bikeshare_df.loc[bikeshare_df['name_from'] == 'Simcoe St / Queen St W','name_from'] = 'Simcoe St / Dundas St W'
bikeshare_df.loc[bikeshare_df['name_to'] == 'Simcoe St / Queen St W','name_to'] = 'Simcoe St / Dundas St W'
bikeshare_df.loc[bikeshare_df['name_from'] == 'Sherbourne St / Wellesley St E','name_from'] = 'Sherbourne St / Wellesley St'
bikeshare_df.loc[bikeshare_df['name_to'] == 'Sherbourne St / Wellesley St E','name_to'] = 'Sherbourne St / Wellesley St'

In [0]:
#exporting dataframe checkpoint
export_csv = bikeshare_df.to_csv(r'/content/drive/My Drive/BikeshareData - City of Toronto/bikeshare_df.csv',index = False)

In [0]:
bikeshare_df = pd.read_csv(r'/content/drive/My Drive/BikeshareData - City of Toronto/bikeshare_df.csv')

#Table Outputs and Aggregations

###Calculating Popular Usage Hours

In [0]:
#determining top usage hours
usagehours_df = bikeshare_df[['trip_start_time','StartHour']].groupby(['StartHour']).count().sort_values('trip_start_time',ascending = False)

In [135]:
usagehours_df.head()

,trip_start_time
StartHour,
17,386518
18,303806
16,294533
8,258620
12,239514


**Table Output:3.1.1.**

In [136]:
usagehours_df.rename(columns={"trip_start_time": "Count of Records"})

,Count of Records
StartHour,
17,386518
18,303806
16,294533
8,258620
12,239514
19,237631
13,232908
21,218550
20,218028


In [0]:
#export table output
export_csv = usagehours_df.to_csv(r'/content/drive/My Drive/BikeshareData - City of Toronto/TableOutput3.1.1.csv',index = False)

###Calculating Most used Locations and Seasonal Variance

In [0]:
#getting number of records per station and sort by descending
most_df = bikeshare_df[['station_id_from','name_from','trip_start_time']].groupby(['station_id_from','name_from']).count().sort_values('trip_start_time',ascending = False)

In [138]:
most_df.head(10)

,,trip_start_time
station_id_from,name_from,
7022.0,Simcoe St / Dundas St W,54353
7033.0,Union Station,51164
7076.0,York St / Queens Quay W,48951
7030.0,Bay St / Wellesley St W,45425
7016.0,Bay St / Queens Quay W (Ferry Terminal),43365
7006.0,Bay St / College St (East Side),42543
7049.0,Queen St W / Portland St,41838
7042.0,Sherbourne St / Wellesley St,39916
7059.0,Front St W / Blue Jays Way,39668


In [0]:
#taking rows only from top 10 most used stations
mostused_df = bikeshare_df[(bikeshare_df['station_id_from'] == 7033) |(bikeshare_df['station_id_from'] == 7076)|(bikeshare_df['station_id_from'] == 7030)|(bikeshare_df['station_id_from'] == 7016)|(bikeshare_df['station_id_from'] == 7006)|(bikeshare_df['station_id_from'] == 7049)|(bikeshare_df['station_id_from'] == 7059)|(bikeshare_df['station_id_from'] == 7022)|(bikeshare_df['station_id_from'] == 7010)|(bikeshare_df['station_id_from'] == 7042)]

In [0]:
#aggregrating most used stations by count and looking at seasonal variance by quarter
top10locations =  mostused_df[['station_id_from','name_from','lat_from','lon_from','Quarter','trip_start_time']].groupby(['station_id_from','name_from','lat_from','lon_from','Quarter']).count()

In [0]:
#renaming column to one that makes sense
top10locations = top10locations.rename(columns = {'trip_start_time':'Number of Records'})

**Table Output :3.1.2.**

In [142]:
top10locations

Number of Records
station_id_from name_from                               lat_from  lon_from   Quarter                   
7006.0          Bay St / College St (East Side)         43.660439 -79.385525 1                     4327
                                                                             2                     9132
                                                                             3                    18354
                                                                             4                    10730
7010.0          King St W / Spadina Ave                 43.645323 -79.395003 1                     3550
                                                                             2                     9808
                                                                             3                    16112
                                                                             4                     7989
7016.0          Bay St / Queens Quay W (Ferry Terminal) 43.640823 -79.376265 1                     1892
                                                                             2                    10505
                                                                             3                    24476
                                                                             4                     6492
7022.0          Simcoe St / Dundas St W                 43.650858 -79.387582 1                     4913
                                                                             2                    11353
                                                                             3                    24752
                                                                             4                    13335
7030.0          Bay St / Wellesley St W                 43.664088 -79.387095 1                     4620
                                                                             2                    10328
                                                                             3                    18811
                                                                             4                    11666
7033.0          Union Station                           43.645609 -79.380386 1                     5200
                                                                             2                    12045
                                                                             3                    21488
                                                                             4                    12431
7042.0          Sherbourne St / Wellesley St            43.667300 -79.374000 1                     4566
                                                                             2                     9099
                                                                             3                    16043
                                                                             4                    10208
7049.0          Queen St W / Portland St                43.647600 -79.401500 1                     3458
                                                                             2                     9811
                                                                             3                    18998
                                                                             4                     9571
7059.0          Front St W / Blue Jays Way              43.643473 -79.390477 1                     3321
                                                                             2                     8935
                                                                             3                    18334
                                                                             4                     9078
7076.0          York St / Queens Quay W                 43.640132 -79.380464 1                     2433
                              

In [0]:
#export table output
export_csv = top10locations.to_csv(r'/content/drive/My Drive/BikeshareData - City of Toronto/TableOutput3.1.2.csv',index = False)

In [0]:
#isolating top 10 location coordinates for mapping
tenbikestations_df = bikestations_df[(bikestations_df['station_id'] == 7033) |(bikestations_df['station_id'] == 7076)|(bikestations_df['station_id'] == 7030)|(bikestations_df['station_id'] == 7016)|(bikestations_df['station_id'] == 7006)|(bikestations_df['station_id'] == 7049)|(bikestations_df['station_id'] == 7059)|(bikestations_df['station_id'] == 7022)|(bikestations_df['station_id'] == 7010)|(bikestations_df['station_id'] == 7042)]

In [0]:
#exporting top locations coordinates 
export_csv = tenbikestations_df.to_csv(r'/content/drive/My Drive/BikeshareData - City of Toronto/tenbikestations.csv')

In [0]:
#export mostused_df for mapping
export_csv = mostused_df.to_csv(r'/content/drive/My Drive/BikeshareData - City of Toronto/mostused_df.csv',index = False)

###Calculating Popular Routes and Seasonal Variance

In [0]:
#aggregating for most used routes by count
idroutes_df = bikeshare_df[['Route Id','trip_start_time']].groupby(['Route Id']).count().sort_values('trip_start_time',ascending = False)

In [145]:
idroutes_df.head(20)

,trip_start_time
Route Id,
7016-7016,7457
7171-7171,5371
7059-7033,4598
7076-7076,4356
7175-7175,3655
7288-7288,2972
7203-7076,2742
7022-7022,2616
7051-7042,2565


Two of Three most trafficked Routes are returning back to their Start Stations

In [0]:
#taking rows only from 3 most used not going back to same station
routes_df = bikeshare_df[(bikeshare_df['Route Id'] == '7051-7042') | (bikeshare_df['Route Id'] == '7203-7076') | (bikeshare_df['Route Id'] == '7059-7033')]

In [0]:
#aggregating counts of records by route and quarter
popular_routes = routes_df[['Route Id','name_from','name_to','Quarter','trip_start_time']].groupby(['Route Id','name_from','name_to','Quarter']).count()

**Table Output:3.1.3.**

took down only routes going to other stations

In [148]:
#renaming column to accurate representation
popular_routes.rename(columns = {'trip_start_time':'Number of Records'})

Number of Records
Route Id  name_from                         name_to                      Quarter                   
7051-7042 Wellesley St E / Yonge St Green P Sherbourne St / Wellesley St 1                      228
                                                                         2                      581
                                                                         3                     1083
                                                                         4                      673
7059-7033 Front St W / Blue Jays Way        Union Station                1                      599
                                                                         2                     1059
                                                                         3                     1710
                                                                         4                     1230
7203-7076 Bathurst St / Queens Quay W       York St / Queens Quay W      1                      101
                                                                         2                      706
                                                                         3                     1373
                                                                         4                      562

In [0]:
#exporting top routes for mapping
export_csv = routes_df.to_csv(r'/content/drive/My Drive/BikeshareData - City of Toronto/routes_df.csv',index = False)

In [0]:
#isolate 6 coordinates for mapping routes
threeroutes_df = bikestations_df[(bikestations_df['station_id'] == 7033) |(bikestations_df['station_id'] == 7076)|(bikestations_df['station_id'] == 7203)|(bikestations_df['station_id'] == 7042)|(bikestations_df['station_id'] == 7051)|(bikestations_df['station_id'] == 7059)]

In [164]:
threeroutes_df

,station_id,name,lat,lon
33,7033.0,Union Station,43.645609,-79.380386
42,7042.0,Sherbourne St / Wellesley St E,43.667300,-79.374000
51,7051.0,Wellesley St E / Yonge St (Green P),43.665060,-79.383570
59,7059.0,Front St W / Blue Jays Way,43.643473,-79.390477
75,7076.0,York St / Queens Quay W,43.640132,-79.380464
189,7203.0,Bathurst St/Queens Quay(Billy Bishop Airport),43.635492,-79.398253


In [0]:
#exporting 3 route coordinates 
export_csv = threeroutes_df.to_csv(r'/content/drive/My Drive/BikeshareData - City of Toronto/threeroutes.csv')

In [0]:
#export table output
export_csv = popular_routes.to_csv(r'/content/drive/My Drive/BikeshareData - City of Toronto/TableOutput3.1.3.csv',index = False)

###Calculate Total Extra Usage Fees by Quarter

**Table Output:3.1.4.**

In [0]:
extrafees_df = bikeshare_df[['Quarter','ExtraFees']].groupby(['Quarter']).sum()

In [152]:
extrafees_df.head()

,ExtraFees
Quarter,
1,27581
2,419435
3,1069795
4,166738


In [0]:
#export table output
export_csv = extrafees_df.to_csv(r'/content/drive/My Drive/BikeshareData - City of Toronto/TableOutput3.1.4.csv',index = False)

#Connecting to Postgres Database 

In [166]:
pip install psycopg2

In [0]:
import psycopg2
connection = psycopg2.connect(
    host = 'database-1.cglpfnjcmmha.us-east-2.rds.amazonaws.com',
    port = 5432,
    user = 'postgres',
    password = 'bikeshare',
    database='torontobikeshare'
    )
cursor=connection.cursor()

In [173]:
type(bikeshare_df['user_type'][0])

str

In [182]:
postgres_df['RouteId'].dtypes

dtype('O')

In [0]:
#changing column name for compatibility with postgres database
postgres_df = bikeshare_df.rename(columns = {'Route Id':'RouteId'})

In [0]:
#export database csv
export_csv = postgres_df.to_csv(r'/content/drive/My Drive/BikeshareData - City of Toronto/postgres_df.csv',index = False)

In [0]:
#creating table bikedata
cursor.execute("""CREATE TABLE bikedata(
trip_start_time timestamp,
trip_stop_time timestamp,
trip_duration_seconds integer,
user_type text,
station_id_from float,
name_from text,
lat_from float,
lon_from float,
station_id_to float,
name_to text,
lat_to float,
lon_to float,
StartHour integer,
StopHour integer,
Quarter integer,
Year integer,
Month integer,
DayofWeek text,
DayNumber integer,
ExtraFees integer,
RouteId text)""")

connection.commit()

In [195]:
#getting schema info
sql = """
SELECT "table_name","column_name", "data_type", "table_schema"
FROM INFORMATION_SCHEMA.COLUMNS
WHERE "table_schema" = 'public'
ORDER BY table_name  
"""
pd.read_sql(sql, con=connection)

,table_name,column_name,data_type,table_schema
0,bikedata,month,integer,public
1,bikedata,lat_to,double precision,public
2,bikedata,trip_stop_time,timestamp without time zone,public
3,bikedata,dayofweek,text,public
4,bikedata,name_to,text,public
5,bikedata,lon_to,double precision,public
6,bikedata,extrafees,integer,public
7,bikedata,daynumber,integer,public
8,bikedata,quarter,integer,public
9,bikedata,station_id_from,double precision,public


In [0]:
#loading csv into database table
with open('/content/drive/My Drive/BikeshareData - City of Toronto/postgres_df.csv', 'r') as row:
    next(row)# Skip the header row.
    cursor.copy_from(row, 'bikedata', sep=',')

connection.commit()

In [194]:
#querying
sql = """
SELECT *
FROM bikedata
limit 5
"""

pd.read_sql(sql, con=connection)

,trip_start_time,trip_stop_time,trip_duration_seconds,user_type,station_id_from,name_from,lat_from,lon_from,station_id_to,name_to,lat_to,lon_to,starthour,stophour,quarter,year,month,dayofweek,daynumber,extrafees,routeid
0,2016-07-09 01:03:00,2016-07-09 01:15:00,714,Member,7100.0,Dundas St E / Regent Park Blvd,43.660207,-79.361275,7095.0,Danforth Ave / Ellerbeck St,43.677076,-79.35667,1,1,3,2016,7,Saturday,9,0,7100-7095
1,2016-09-03 22:37:00,2016-09-03 22:48:00,685,Member,7100.0,Dundas St E / Regent Park Blvd,43.660207,-79.361275,7095.0,Danforth Ave / Ellerbeck St,43.677076,-79.35667,22,22,3,2016,9,Saturday,3,0,7100-7095
2,2016-08-05 16:21:00,2016-08-05 16:33:00,718,Member,7100.0,Dundas St E / Regent Park Blvd,43.660207,-79.361275,7095.0,Danforth Ave / Ellerbeck St,43.677076,-79.35667,16,16,3,2016,8,Friday,5,0,7100-7095
3,2016-09-23 10:15:00,2016-09-23 10:28:00,792,Member,7100.0,Dundas St E / Regent Park Blvd,43.660207,-79.361275,7095.0,Danforth Ave / Ellerbeck St,43.677076,-79.35667,10,10,3,2016,9,Friday,23,0,7100-7095
4,2016-10-15 13:40:00,2016-10-15 13:51:00,653,Member,7100.0,Dundas St E / Regent Park Blvd,43.660207,-79.361275,7095.0,Danforth Ave / Ellerbeck St,43.677076,-79.35667,13,13,4,2016,10,Saturday,15,0,7100-7095
